In [62]:
import os
import requests
import pandas as pd

In [6]:
os.listdir('archive/food-101/food-101/images')

['foie_gras',
 'club_sandwich',
 'cheese_plate',
 'cup_cakes',
 'garlic_bread',
 'gnocchi',
 'ice_cream',
 'samosa',
 'donuts',
 'tuna_tartare',
 'filet_mignon',
 '.DS_Store',
 'seaweed_salad',
 'french_toast',
 'chicken_curry',
 'shrimp_and_grits',
 'steak',
 'cheesecake',
 'red_velvet_cake',
 'waffles',
 'churros',
 'gyoza',
 'lobster_roll_sandwich',
 'huevos_rancheros',
 'breakfast_burrito',
 'grilled_cheese_sandwich',
 'spaghetti_bolognese',
 'falafel',
 'poutine',
 'greek_salad',
 'beef_tartare',
 'fried_calamari',
 'guacamole',
 'ravioli',
 'lobster_bisque',
 'beet_salad',
 'risotto',
 'crab_cakes',
 'strawberry_shortcake',
 'edamame',
 'ceviche',
 'hot_and_sour_soup',
 'spring_rolls',
 'sashimi',
 'paella',
 'clam_chowder',
 'miso_soup',
 'escargots',
 'hot_dog',
 'pulled_pork_sandwich',
 'bruschetta',
 'panna_cotta',
 'fish_and_chips',
 'pad_thai',
 'tiramisu',
 'takoyaki',
 'macarons',
 'apple_pie',
 'cannoli',
 'scallops',
 'frozen_yogurt',
 'chicken_quesadilla',
 'mussels',


In [104]:
# setting api to USDA FoodData Central

API_KEY = "0fzhN6pJrGh1g812ETw7ptA0gILWEcO7IOhJjexP"
BASE_URL = "https://api.nal.usda.gov/fdc/v1"

def search_food(food_name):
    """
    Searches for a food item using the USDA API and retrieves its basic information.

    Args:
        food_name (str): Name of the food to search for.

    Returns:
        dict: Data of the first food result, or None if not found.
    """
    url = f"{BASE_URL}/foods/search"
    headers = {"X-Api-Key": API_KEY}
    params = {
        "query": food_name,
        "pageSize": 1  # Limit results to the first match
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("foods"):
            return data["foods"][0]  # Return the first result
    return None

def get_calories(food_name):
    """
    Fetches calorie data and serving size for a specific food item by name.

    Args:
        food_name (str): Name of the food to fetch calories and serving size for.

    Returns:
        tuple: A tuple containing the calorie content (float) and serving size (float), or None if not found.
    """
    food_data = search_food(food_name)
    
    # Check if food_data is valid and contains the fdcId
    if food_data:
        fdc_id = food_data.get("fdcId")
        url = f"{BASE_URL}/food/{fdc_id}"
        headers = {"X-Api-Key": API_KEY}
        
        # Make the API request
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            details = response.json()
            #print(details)
            calories = None
            serving_size = None
            protein = None
            food_nutrients = details.get("foodNutrients", [])
            for nutrient in food_nutrients:
                if (nutrient.get("nutrient", {}).get("name") == "Energy") and (nutrient.get("nutrient", {}).get("unitName") == "kcal"):
                    calories = nutrient.get("amount")
                if (nutrient.get("nutrient", {}).get("name") == "Protein") and (nutrient.get("nutrient", {}).get("unitName") == "g"):
                    protein = nutrient.get("amount")
            
            # Get the serving size if available
            serving_size = details.get("servingSize", 100)
            
            return calories, serving_size, protein
    
    return None, 100, None

In [105]:
print(get_calories("pancakes"))

(274.0, 62.0, 6.45)


In [111]:
food_list = [food.replace('_', ' ') for food in os.listdir('archive/food-101/food-101/images')]

data = []

# Fetch the calories and serving size for each food item and store the results in the data list
for food in food_list:
    calories, serving_size, protein = get_calories(food)
    data.append({
        "Food": food.replace('_', ' ').title(),
        "Calories (kcal)": calories,
        "Serving Size (g)": serving_size,
        "Protein (g)": protein
    })

In [112]:
# Convert the collected data to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df

,Food,Calories (kcal),Serving Size (g),Protein (g)
0,Foie Gras,462.0,100.0,11.40
1,Club Sandwich,220.0,100.0,16.00
2,Cheese Plate,192.0,100.0,6.46
3,Cup Cakes,356.0,45.0,2.22
4,Garlic Bread,351.0,57.0,8.77
...,...,...,...,...
97,Baklava,440.0,100.0,6.58
98,Creme Brulee,113.0,100.0,4.06
99,Carrot Cake,424.0,85.0,2.35
100,Onion Rings,500.0,28.0,7.14


In [113]:
df[df.isnull().any(axis=1)]

,Food,Calories (kcal),Serving Size (g),Protein (g)
55,Takoyaki,NaN,100.0,NaN


In [114]:
df.loc[df["Food"] == "Takoyaki", ["Calories (kcal)", "Serving Size (g)", "Protein (g)"]] = [70.0, 40.0, 3]

In [115]:
df[df.isnull().any(axis=1)]

,Food,Calories (kcal),Serving Size (g),Protein (g)


In [117]:
df.to_csv('calories.csv', index=False)